<a href="https://colab.research.google.com/github/nitingarg2015/EVA8/blob/master/Assignment%204/Assignment_4_Iteration_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# use dataloader to create training and test datasets with define batch size

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
# CNN model definition
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),                      #input: 1*28*28, output: 8*28*28, RF: 3*3
            nn.BatchNorm2d(8),
            nn.Dropout(0.05)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),                     #input: 8*28*28, output: 16*28*28, RF: 5*5 
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 20, 3, padding=1),                     #input: 16*28*28, output: 20*28*28, RF: 7*7 
            nn.BatchNorm2d(20),
            nn.Dropout(0.05)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(20, 8, 1),                                #input: 20*28*28, output: 8*28*28, RF: 7*7
            nn.MaxPool2d(2, 2),                                  #input: 8*28*28, output: 8*14*14, RF: 8*8
            nn.Dropout(0.05)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(8, 8, 3, padding=1),                     #input: 8*14*14, output: 8*14*14, RF: 12*12 
            nn.BatchNorm2d(8),
            nn.Dropout(0.05)
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),                     #input: 8*14*14, output: 16*14*14, RF: 16*16 
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(16, 8, 1),                                 #input: 16*14*14, output: 8*14*14, RF: 16*16 
            nn.MaxPool2d(2, 2),                                  #input: 8*14*14, output: 8*7*7, RF: 18*18
            #nn.Dropout(0.05)
        )

        self.conv8 = nn.Sequential(
            nn.Conv2d(8, 8, 3, padding = 1),                     #input: 8*7*7, output: 8*7*7, RF: 26*26 
            nn.BatchNorm2d(8),
            nn.Dropout(0.05)
        )

        self.conv9 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding = 1),                    #input: 8*7*7, output: 16*7*7, RF: 34*34 
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )

        self.conv10 = nn.Sequential(
            nn.AvgPool2d(2, 2),                                  #input: 16*7*7, output: 16*3*3, RF: 38*38
            nn.Conv2d(16, 10, 3),                                #input: 16*3*3, output: 10*1*1, RF: 54*54
            #nn.BatchNorm2d(16),
            #nn.Dropout(0.10)
        )
                
        
    def forward(self, x):
        x = F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = F.relu(self.conv6(F.relu(self.conv5(F.relu(self.conv4(x))))))
        x = F.relu(self.conv9(F.relu(self.conv8(F.relu(self.conv7(x))))))

        x = self.conv10(x)
        x = x.view(x.size(0), -1)

        #x = self.fc(x)
        
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
#instantiate Net class and check model summary

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 20, 28, 28]           2,900
       BatchNorm2d-8           [-1, 20, 28, 28]              40
           Dropout-9           [-1, 20, 28, 28]               0
           Conv2d-10            [-1, 8, 28, 28]             168
        MaxPool2d-11            [-1, 8, 14, 14]               0
          Dropout-12            [-1, 8, 14, 14]               0
           Conv2d-13            [-1, 8, 14, 14]             584
      BatchNorm2d-14            [-1, 8,

In [ ]:
# Set for training on GPU
#define optimizer
#set the loop for training in epochs
from torch.optim.lr_scheduler import StepLR # Import your choice of scheduler here

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.2)

for epoch in range(1, 16):
  print("EPOCH:", epoch, "Learning Rate: ", optimizer.param_groups[0]["lr"])
  
  train(model, device, train_loader, optimizer, epoch)
  scheduler.step()
  test(model, device, test_loader)

EPOCH: 1 Learning Rate:  0.1


Loss=0.05808623507618904 Batch_id=468 Accuracy=93.97: 100%|██████████| 469/469 [00:17<00:00, 27.30it/s]



Test set: Average loss: 0.0652, Accuracy: 9780/10000 (97.80%)

EPOCH: 2 Learning Rate:  0.1


Loss=0.054971933364868164 Batch_id=468 Accuracy=97.59: 100%|██████████| 469/469 [00:18<00:00, 25.28it/s]



Test set: Average loss: 0.0667, Accuracy: 9804/10000 (98.04%)

EPOCH: 3 Learning Rate:  0.1


...


Test set: Average loss: 0.0213, Accuracy: 9934/10000 (99.34%)

EPOCH: 14 Learning Rate:  0.0008000000000000003


Loss=0.005238950718194246 Batch_id=468 Accuracy=99.09: 100%|██████████| 469/469 [00:18<00:00, 25.94it/s]



Test set: Average loss: 0.0214, Accuracy: 9932/10000 (99.32%)

EPOCH: 15 Learning Rate:  0.0008000000000000003


Loss=0.03668414428830147 Batch_id=468 Accuracy=99.10: 100%|██████████| 469/469 [00:16<00:00, 28.13it/s]



Test set: Average loss: 0.0213, Accuracy: 9933/10000 (99.33%)

